In [ ]:
import os
import subprocess
import sys

# Safely run pip install without noisy output
subprocess.run(
    [sys.executable, "-m", "pip", "install", "py3Dmol", "ipywidgets"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# 📚 Import libraries
import py3Dmol
import requests
import ipywidgets as widgets
from IPython.display import display, HTML, FileLink, clear_output
from IPython.display import Image as IPImage

# 📂 Directory where multiple CIF models are stored
model_dir = "protein_fold_model"
available_models = sorted([
    f for f in os.listdir(model_dir) if f.endswith(".cif")
])

# Helper: Map temperature to available model index
def get_model_path_from_temperature(temp: int) -> str:
    model_idx = min(len(available_models)-1, abs(temp) // 10)
    return os.path.join(model_dir, available_models[model_idx])

# 🎨 Enhanced UI Styling with Modern Design
custom_css = """
<style>
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700;800&family=JetBrains+Mono:wght@400;500&display=swap');

:root {
    --primary-blue: #2563eb;
    --primary-blue-dark: #1d4ed8;
    --secondary-purple: #7c3aed;
    --accent-cyan: #06b6d4;
    --success-green: #10b981;
    --warning-amber: #f59e0b;
    --neutral-50: #f9fafb;
    --neutral-100: #f3f4f6;
    --neutral-200: #e5e7eb;
    --neutral-300: #d1d5db;
    --neutral-600: #4b5563;
    --neutral-700: #374151;
    --neutral-800: #1f2937;
    --neutral-900: #111827;
    --glass-bg: rgba(255, 255, 255, 0.1);
    --glass-border: rgba(255, 255, 255, 0.2);
}

* {
    box-sizing: border-box;
}

.polar-container {
    font-family: 'Inter', -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 50%, #667eea 100%);
    background-size: 400% 400%;
    animation: gradientShift 15s ease infinite;
    border-radius: 24px;
    padding: 3rem 2rem;
    box-shadow: 
        0 25px 50px -12px rgba(0, 0, 0, 0.25),
        0 0 0 1px rgba(255, 255, 255, 0.1);
    margin: 1rem 0 2rem 0;
    position: relative;
    overflow: hidden;
    backdrop-filter: blur(20px);
}

@keyframes gradientShift {
    0%, 100% { background-position: 0% 50%; }
    50% { background-position: 100% 50%; }
}

.polar-container::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    bottom: 0;
    background: url('data:image/svg+xml,<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 100 100"><defs><pattern id="snowflakes" x="0" y="0" width="25" height="25" patternUnits="userSpaceOnUse"><circle cx="3" cy="3" r="1.2" fill="rgba(255,255,255,0.08)"/><circle cx="15" cy="10" r="0.8" fill="rgba(255,255,255,0.12)"/><circle cx="22" cy="18" r="1" fill="rgba(255,255,255,0.06)"/><circle cx="8" cy="20" r="0.6" fill="rgba(255,255,255,0.1)"/></pattern></defs><rect width="100" height="100" fill="url(%23snowflakes)"/></svg>') repeat;
    pointer-events: none;
    animation: snow 25s linear infinite;
    opacity: 0.7;
}

@keyframes snow {
    0% { transform: translateY(-100px) rotate(0deg); }
    100% { transform: translateY(100vh) rotate(360deg); }
}

.polar-container::after {
    content: '';
    position: absolute;
    top: -50%;
    left: -50%;
    width: 200%;
    height: 200%;
    background: radial-gradient(circle, rgba(255,255,255,0.1) 0%, transparent 70%);
    animation: aurora 20s linear infinite;
    pointer-events: none;
}

@keyframes aurora {
    0%, 100% { transform: rotate(0deg) scale(1); opacity: 0.3; }
    33% { transform: rotate(120deg) scale(1.1); opacity: 0.6; }
    66% { transform: rotate(240deg) scale(0.9); opacity: 0.4; }
}

.polar-title {
    color: white;
    font-size: 3rem;
    font-weight: 800;
    text-align: center;
    margin: 0 0 1rem 0;
    text-shadow: 
        0 4px 8px rgba(0,0,0,0.3),
        0 2px 4px rgba(0,0,0,0.2);
    position: relative;
    z-index: 1;
    letter-spacing: -0.02em;
}

.polar-subtitle {
    color: rgba(255,255,255,0.95);
    font-size: 1.25rem;
    text-align: center;
    margin: 0 0 2rem 0;
    font-weight: 400;
    position: relative;
    z-index: 1;
    letter-spacing: 0.01em;
}

.controls-panel {
    background: linear-gradient(135deg, 
        rgba(255,255,255,0.95) 0%, 
        rgba(248,250,252,0.98) 100%);
    backdrop-filter: blur(20px);
    border-radius: 20px;
    padding: 2rem;
    margin: 1.5rem 0;
    box-shadow: 
        0 20px 25px -5px rgba(0, 0, 0, 0.1),
        0 10px 10px -5px rgba(0, 0, 0, 0.04),
        0 0 0 1px rgba(255, 255, 255, 0.5);
    position: relative;
    z-index: 1;
    border: 1px solid rgba(255, 255, 255, 0.3);
}

.controls-panel::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    height: 3px;
    background: linear-gradient(90deg, var(--primary-blue), var(--secondary-purple), var(--accent-cyan));
    border-radius: 20px 20px 0 0;
}

.viewer-container {
    background: linear-gradient(135deg, 
        rgba(255,255,255,0.98) 0%, 
        rgba(248,250,252,0.95) 100%);
    backdrop-filter: blur(20px);
    border-radius: 20px;
    padding: 1.5rem;
    margin: 1.5rem 0;
    box-shadow: 
        0 20px 25px -5px rgba(0, 0, 0, 0.1),
        0 10px 10px -5px rgba(0, 0, 0, 0.04),
        inset 0 1px 0 rgba(255, 255, 255, 0.6);
    min-height: 580px;
    position: relative;
    z-index: 1;
    border: 1px solid rgba(255, 255, 255, 0.4);
}

.info-panel {
    background: linear-gradient(135deg, 
        rgba(255,255,255,0.95) 0%, 
        rgba(240,249,255,0.98) 50%,
        rgba(245,243,255,0.95) 100%);
    backdrop-filter: blur(20px);
    border-radius: 20px;
    padding: 2rem;
    margin: 1.5rem 0;
    box-shadow: 
        0 20px 25px -5px rgba(0, 0, 0, 0.1),
        0 10px 10px -5px rgba(0, 0, 0, 0.04),
        0 0 0 1px rgba(255, 255, 255, 0.5);
    position: relative;
    z-index: 1;
    border: 1px solid rgba(255, 255, 255, 0.3);
}

.info-panel::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    height: 3px;
    background: linear-gradient(90deg, var(--success-green), var(--accent-cyan), var(--secondary-purple));
    border-radius: 20px 20px 0 0;
}

.protein-title {
    color: var(--primary-blue-dark);
    font-size: 1.5rem;
    font-weight: 700;
    margin-bottom: 1.5rem;
    display: flex;
    align-items: center;
    gap: 0.75rem;
    padding-bottom: 0.5rem;
    border-bottom: 2px solid rgba(37, 99, 235, 0.1);
}

.protein-description {
    color: var(--neutral-700);
    line-height: 1.8;
    font-size: 1rem;
    font-weight: 400;
}

.protein-description strong {
    color: var(--neutral-800);
    font-weight: 600;
}

.temperature-display {
    background: linear-gradient(135deg, var(--primary-blue), var(--primary-blue-dark));
    color: white;
    padding: 1rem 2rem;
    border-radius: 16px;
    font-weight: 600;
    text-align: center;
    margin: 1rem 0;
    box-shadow: 
        0 10px 15px -3px rgba(37, 99, 235, 0.4),
        0 4px 6px -2px rgba(37, 99, 235, 0.2);
    font-size: 1.1rem;
    letter-spacing: 0.01em;
    position: relative;
    overflow: hidden;
}

.temperature-display::before {
    content: '';
    position: absolute;
    top: 0;
    left: -100%;
    width: 100%;
    height: 100%;
    background: linear-gradient(90deg, transparent, rgba(255,255,255,0.2), transparent);
    animation: shimmer 3s infinite;
}

@keyframes shimmer {
    0% { left: -100%; }
    100% { left: 100%; }
}

.loading-spinner {
    border: 4px solid rgba(37, 99, 235, 0.2);
    border-radius: 50%;
    border-top: 4px solid var(--primary-blue);
    width: 40px;
    height: 40px;
    animation: spin 1s linear infinite;
    margin: 2rem auto;
    position: relative;
}

.loading-spinner::after {
    content: '';
    position: absolute;
    top: 2px;
    left: 2px;
    right: 2px;
    bottom: 2px;
    border-radius: 50%;
    border: 2px solid transparent;
    border-top-color: var(--accent-cyan);
    animation: spin 0.8s linear infinite reverse;
}

@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}

.status-message {
    background: linear-gradient(135deg, var(--success-green), #059669);
    color: white;
    padding: 1rem 1.5rem;
    border-radius: 12px;
    margin: 1rem 0;
    font-weight: 600;
    box-shadow: 
        0 10px 15px -3px rgba(16, 185, 129, 0.4),
        0 4px 6px -2px rgba(16, 185, 129, 0.2);
    font-size: 1rem;
    position: relative;
    overflow: hidden;
}

.status-message::before {
    content: '';
    position: absolute;
    top: 0;
    left: -100%;
    width: 100%;
    height: 100%;
    background: linear-gradient(90deg, transparent, rgba(255,255,255,0.2), transparent);
    animation: shimmer 2s infinite;
}

.controls-title {
    margin: 0 0 1.5rem 0; 
    color: var(--primary-blue-dark); 
    font-weight: 700;
    font-size: 1.3rem;
    display: flex;
    align-items: center;
    gap: 0.5rem;
}

.controls-description {
    color: var(--neutral-600); 
    margin: 0 0 1.5rem 0; 
    font-size: 1rem;
    line-height: 1.6;
}

.enhanced-button {
    background: linear-gradient(135deg, var(--primary-blue), var(--primary-blue-dark)) !important;
    color: white !important;
    border: none !important;
    padding: 14px 28px !important;
    border-radius: 12px !important;
    font-weight: 600 !important;
    font-size: 1rem !important;
    cursor: pointer !important;
    box-shadow: 
        0 10px 15px -3px rgba(37, 99, 235, 0.4),
        0 4px 6px -2px rgba(37, 99, 235, 0.2) !important;
    transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1) !important;
    position: relative !important;
    overflow: hidden !important;
    letter-spacing: 0.01em !important;
}

.enhanced-button:hover {
    transform: translateY(-2px) !important;
    box-shadow: 
        0 20px 25px -5px rgba(37, 99, 235, 0.4),
        0 10px 10px -5px rgba(37, 99, 235, 0.3) !important;
}

.enhanced-button:active {
    transform: translateY(0px) !important;
}

.enhanced-button::before {
    content: '';
    position: absolute;
    top: 0;
    left: -100%;
    width: 100%;
    height: 100%;
    background: linear-gradient(90deg, transparent, rgba(255,255,255,0.2), transparent);
    transition: left 0.5s;
}

.enhanced-button:hover::before {
    left: 100%;
}

.download-button-enhanced {
    background: linear-gradient(135deg, var(--success-green), #059669) !important;
    color: white !important;
    border: none !important;
    padding: 14px 24px !important;
    border-radius: 12px !important;
    font-weight: 600 !important;
    cursor: pointer !important;
    box-shadow: 
        0 10px 15px -3px rgba(16, 185, 129, 0.4),
        0 4px 6px -2px rgba(16, 185, 129, 0.2) !important;
    transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1) !important;
    text-decoration: none !important;
    display: inline-block !important;
    font-size: 1rem !important;
    letter-spacing: 0.01em !important;
}

.download-button-enhanced:hover {
    transform: translateY(-2px) !important;
    box-shadow: 
        0 20px 25px -5px rgba(16, 185, 129, 0.4),
        0 10px 10px -5px rgba(16, 185, 129, 0.3) !important;
    color: white !important;
    text-decoration: none !important;
}

.slider-container {
    background: rgba(255, 255, 255, 0.5);
    padding: 1rem 1.5rem;
    border-radius: 12px;
    border: 1px solid rgba(255, 255, 255, 0.6);
    backdrop-filter: blur(10px);
}

.experimental-section {
    background: linear-gradient(135deg, 
        rgba(255,255,255,0.95) 0%, 
        rgba(240,249,255,0.98) 50%,
        rgba(245,243,255,0.95) 100%);
    backdrop-filter: blur(20px);
    border-radius: 20px;
    padding: 2rem;
    margin: 2rem 0;
    box-shadow: 
        0 20px 25px -5px rgba(0, 0, 0, 0.1),
        0 10px 10px -5px rgba(0, 0, 0, 0.04),
        0 0 0 1px rgba(255, 255, 255, 0.5);
    position: relative;
    z-index: 1;
    border: 1px solid rgba(255, 255, 255, 0.3);
}

.experimental-section::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    height: 3px;
    background: linear-gradient(90deg, var(--warning-amber), var(--accent-cyan), var(--secondary-purple));
    border-radius: 20px 20px 0 0;
}

.experimental-title {
    text-align: center; 
    margin: 0 0 2rem 0;
    color: var(--neutral-800);
    font-weight: 700;
    font-size: 1.75rem;
    position: relative;
}

.experimental-title::after {
    content: '';
    position: absolute;
    bottom: -0.5rem;
    left: 50%;
    transform: translateX(-50%);
    width: 60px;
    height: 3px;
    background: linear-gradient(90deg, var(--primary-blue), var(--secondary-purple));
    border-radius: 2px;
}

.chart-container {
    background: rgba(255, 255, 255, 0.8);
    border-radius: 16px;
    padding: 1.5rem;
    margin: 1rem;
    box-shadow: 
        0 10px 15px -3px rgba(0, 0, 0, 0.08),
        0 4px 6px -2px rgba(0, 0, 0, 0.04);
    border: 1px solid rgba(255, 255, 255, 0.6);
    transition: transform 0.3s ease, box-shadow 0.3s ease;
}

.chart-container:hover {
    transform: translateY(-4px);
    box-shadow: 
        0 20px 25px -5px rgba(0, 0, 0, 0.12),
        0 10px 10px -5px rgba(0, 0, 0, 0.08);
}

.chart-title {
    text-align: center;
    font-weight: 600;
    color: var(--neutral-800);
    margin-bottom: 1rem;
    font-size: 1.1rem;
}

.chart-insight {
    text-align: center; 
    font-size: 0.9rem; 
    color: var(--neutral-600);
    margin-top: 1rem;
    line-height: 1.5;
}

.main-layout {
    padding: 1.5rem;
    max-width: 1400px;
    margin: 0 auto;
}

.viewer-title {
    margin: 0 0 1.5rem 0; 
    color: var(--primary-blue-dark); 
    font-weight: 700; 
    text-align: center;
    font-size: 1.4rem;
    position: relative;
}

.viewer-title::after {
    content: '';
    position: absolute;
    bottom: -0.5rem;
    left: 50%;
    transform: translateX(-50%);
    width: 40px;
    height: 2px;
    background: var(--primary-blue);
    border-radius: 2px;
}

@media (max-width: 768px) {
    .polar-title {
        font-size: 2.2rem;
    }
    
    .polar-subtitle {
        font-size: 1rem;
    }
    
    .controls-panel, .info-panel, .viewer-container {
        padding: 1.5rem;
        margin: 1rem 0;
    }
    
    .main-layout {
        padding: 1rem;
    }
}
</style>
"""

# 🎛️ Enhanced UI Components
title_html = custom_css + """
<div class="polar-container">
    <h1 class="polar-title">🐻‍❄️ PolarBear SK-LAB</h1>
    <p class="polar-subtitle">Advanced Protein Structure Visualization Platform</p>
</div>
"""

title = widgets.HTML(title_html)

run_button = widgets.Button(
    description="🚀 Launch 3D Visualization", 
    button_style='',
    layout=widgets.Layout(height='50px', width='auto')
)
run_button.add_class('enhanced-button')

temp_slider = widgets.IntSlider(
    value=0, 
    min=-40, 
    max=0, 
    step=10, 
    description="🌡️ Temperature:",
    style={'description_width': '120px'},
    layout=widgets.Layout(width='350px')
)

download_html = f"""
<div style="display: inline-block;">
    <a download='fold_2025_06_02_17_04_model_1.cif' href='fold_2025_06_02_17_04_model_1.cif' target='_blank' class="download-button-enhanced">
        📥 Download Structure (.cif)
    </a>
</div>
"""
download_button = widgets.HTML(download_html)

# Enhanced control panel
controls_info = widgets.VBox([
    widgets.HTML("""
        <div class="controls-panel">
            <h3 class="controls-title">🎛️ Visualization Controls</h3>
            <p class="controls-description">
                Adjust the temperature to explore different protein conformations under Arctic conditions. Each temperature setting reveals unique structural adaptations that enable polar bear survival in extreme environments.
            </p>
            <div class="slider-container">
            </div>
        </div>
    """),
    widgets.HBox([
        temp_slider, 
        run_button, 
        download_button
    ], layout=widgets.Layout(
        justify_content='space-between', 
        align_items='center',
        gap="1.5rem",
        margin="1rem 0 0 0"
    ))
])

protein_info_html = """
<div class="info-panel">
    <div class="protein-title">
        🧬 Aquaporin-3 (AQP3) - Arctic Adaptation Protein
    </div>
    <div class="protein-description">
        <strong>Functional Overview:</strong> Aquaporin-3 is a sophisticated membrane channel protein that facilitates the selective transport of water and small solutes including glycerol across cellular membranes.
        <br><br>
        <strong>Arctic Specialization:</strong> In the extreme Arctic environment inhabited by polar bears, AQP3 plays a crucial role in cellular survival mechanisms. The protein helps maintain optimal cellular hydration and membrane fluidity at subzero temperatures by precisely regulating intracellular glycerol concentrations.
        <br><br>
        <strong>Cryoprotective Function:</strong> Glycerol acts as a natural antifreeze compound (cryoprotectant), and AQP3's expression in polar bear skin and hair follicles contributes to remarkable ice resistance by minimizing ice crystal nucleation and preserving cellular structural integrity under severe freezing stress.
        <br><br>
        <em>🔬 Use the temperature slider above to observe how protein conformation adapts to different Arctic conditions.</em>
    </div>
</div>
"""
protein_info = widgets.HTML(protein_info_html)

# 📊 Section: Experimental Graphics Viewer
def create_graphics_section():
    chart_configs = [
        {
            "title": "Lipid Class Abundance",
            "endpoint": "/api/lipid/lipid_chart",
            "insight": "Lipid class distribution in polar bear samples."
        },
        {
            "title": "Fatty Acid Abundance (Bar)",
            "endpoint": "/api/abundance/fa_bar_chart",
            "insight": "Fatty acids vary with environmental exposure."
        },
        {
            "title": "Fatty Acid Composition (Pie)",
            "endpoint": "/api/abundance/fa_pie_chart",
            "insight": "Most abundant fatty acids shown proportionally."
        },
        {
            "title": "NMR Quantification (Stacked Bar)",
            "endpoint": "/api/nmr-quant/stacked-bar",
            "insight": "Relative lipid concentrations per sample."
        },
        {
            "title": "NMR Quantification (Heatmap)",
            "endpoint": "/api/nmr-quant/heatmap",
            "insight": "Heatmap reveals concentration trends."
        }
    ]
    base_url = "http://127.0.0.1:5000"
    chart_widgets = []

    for chart in chart_configs:
        url = f"{base_url}{chart['endpoint']}"
        try:
            resp = requests.get(url)
            if resp.status_code == 200:
                img = widgets.Image(
                    value=resp.content,
                    format='png',
                    layout=widgets.Layout(max_width='100%')
                )
                chart_widgets.append(
                    widgets.VBox([
                        widgets.HTML(f"<h4 class='chart-title'>{chart['title']}</h4>"),
                        img,
                        widgets.HTML(f"<p class='chart-insight'>{chart['insight']}</p>")
                    ], layout=widgets.Layout(margin='0'))
                )
            else:
                chart_widgets.append(widgets.HTML(f"<div class='chart-container'><p style='color:red; text-align:center;'>Failed: {chart['title']} (HTTP {resp.status_code})</p></div>"))
        except Exception as e:
            chart_widgets.append(widgets.HTML(f"<div class='chart-container'><p style='color:red; text-align:center;'>Error: {chart['title']}<br>{str(e)}</p></div>"))

    # Wrap charts in enhanced containers
    enhanced_charts = []
    for widget in chart_widgets:
        enhanced_chart = widgets.HTML(f'<div class="chart-container">{widget.children[0].value if hasattr(widget.children[0], "value") else ""}</div>')
        if hasattr(widget, 'children') and len(widget.children) > 1:
            enhanced_chart = widgets.VBox([
                widget.children[0],  # title
                widgets.HTML('<div class="chart-container">'),
                widget.children[1],  # image
                widgets.HTML('</div>'),
                widget.children[2]   # insight
            ])
        enhanced_charts.append(enhanced_chart)

    first_row = widgets.HBox(enhanced_charts[:3], layout=widgets.Layout(justify_content='space-around', gap='1rem'))
    second_row = widgets.HBox(enhanced_charts[3:], layout=widgets.Layout(justify_content='center', gap='2rem'))

    return widgets.VBox([
        widgets.HTML('<div class="experimental-section">'),
        widgets.HTML("<h2 class='experimental-title'>📈 Experimental Insights</h2>"),
        first_row,
        second_row,
        widgets.HTML('</div>')
    ])

# 🧪 Output area for 3D viewer
viewer_output = widgets.Output()

# Temperature status display
temp_status = widgets.HTML()

def update_temp_status(temp):
    status_html = f"""
    <div class="temperature-display">
        🌡️ Current Temperature: {temp}°C | Arctic Simulation Mode: {'Extreme' if temp <= -30 else 'Moderate' if temp <= -20 else 'Mild'}
    </div>
    """
    temp_status.value = status_html

# ▶️ Enhanced render function with loading states
def on_run_click(b):
    with viewer_output:
        clear_output()
        
        # Show loading
        print("🔄 Initializing 3D molecular visualization...")
        loading_html = '<div class="loading-spinner"></div><p style="text-align: center; color: #6b7280; font-weight: 500; margin-top: 1rem;">Loading protein structure...</p>'
        display(HTML(loading_html))
        
        temp = temp_slider.value
        cif_path = get_model_path_from_temperature(temp)
        
        # Update temperature status
        update_temp_status(temp)
        
        # Read CIF content
        with open(cif_path, "r") as f:
            cif_str = f.read()
        
        # Clear loading and show viewer
        clear_output()
        display(HTML('<h3 style="margin: 0 0 1rem 0; color: #1e40af; font-weight: 600; text-align: center;">🔬 3D Molecular Structure Viewer</h3>'))

        # Visualize with enhanced styling
        view = py3Dmol.view(width='100%', height=500)
        view.addModel(cif_str, "cif")
        view.setStyle({"cartoon": {"color": "spectrum"}})
        view.setBackgroundColor("#f8fafc")
        view.zoomTo()
        view.show()
        
        # Status message
        status_html = f"""
        <div class="status-message">
            ✅ Successfully loaded: {os.path.basename(cif_path)} | Temperature: {temp}°C
        </div>
        """
        display(HTML(status_html))

# Interactive temperature updates
def on_temp_change(change):
    update_temp_status(change['new'])

temp_slider.observe(on_temp_change, names='value')

run_button.on_click(on_run_click)

# Initialize temperature status
update_temp_status(0)

# 🧱 Enhanced Layout with better spacing and organization
layout = widgets.VBox([
    title,
    viewer_output,             # 👈 Pindahkan ini ke atas
    temp_status,
    controls_info,             # 👈 Ini berisi slider+button+download
    protein_info,
    create_graphics_section()
], layout=widgets.Layout(padding='1rem'))


display(layout)